# Analysis of Musical and Lyrical Trends at Intelligent Interactive Systems (MIIS)

#### Oktay Ozan Güner   -  ID : OZAN_ID
#### Juan Miguel Alfonso Habana   -  ID : MIGUEL_ID

# Introduction

Since 1990, the way people interact with music has evolved significantly. This period marks a transition from the tangible, physical media of CDs and vinyl to the intangible, yet infinitely accessible world of digital music. 
* How digitalization and streaming have influenced listeners? 
* How the listening habits have changed over time?

We'll examine how our music listening habits have been affected from duration of the songs to the way sentiment of the lyrics.


In [1]:
import numpy as np
import pandas as pd

import spacy
import asent

import tweetnlp

from tqdm import tqdm


tqdm.pandas()

In [ ]:
lyrics = pd.read_csv('./Final_Data_20240309.csv')

In [ ]:
class SentimentAnalyzer():
  def __init__(self):
    self.nlp = spacy.blank('en')
    self.nlp.add_pipe('sentencizer')
    self.nlp.add_pipe('asent_en_v1')

    self.model = tweetnlp.load_model('sentiment')

  def extract_sentiment_scores(self,text):
    doc = self.nlp(text)
    output = {'ascent_'+ output.split('=')[0]:output.split('=')[1] for output in str(doc._.polarity).split(' ')}
    sentiment = self.model.sentiment(text, return_probability=True)
    output['tweetnlp_label'] = sentiment['label']
    output['tweetnlp_neg'] = sentiment['probability']['negative']
    output['tweetnlp_neu'] = sentiment['probability']['neutral']
    output['tweetnlp_pos'] = sentiment['probability']['positive']

    return output

In [ ]:
sa = SentimentAnalyzer()

lyrics_sentiment = lyrics.progress_apply(lambda x: sa.extract_sentiment_scores(x['Translated_Lyrics_2']), axis='columns', result_type='expand')
lyrics = pd.concat([lyrics, lyrics_sentiment], axis='columns')
lyrics = lyrics.drop(labels='Unnamed: 0', axis=1)
lyrics.to_csv('lyrics_sentiment.csv', index=False)
